# Matrix factorization

There is probably no need to say that there is too much information on the Web nowadays. Search engines help us a little bit. What is better is to have something interesting recommended to us automatically without asking. Indeed, from as simple as a list of the most popular questions and answers on Quora to some more personalized recommendations we received on Amazon, we are usually offered recommendations on the Web.

Recommendations can be generated by a wide range of algorithms. While user-based or item-based collaborative filtering methods are simple and intuitive, matrix factorization techniques are usually more effective because they allow us to discover the latent features underlying the interactions between users and items. Of course, matrix factorization is simply a mathematical tool for playing around with matrices, and is therefore applicable in many scenarios where one would like to find out something hidden under the data.

In this tutorial, we will go through the basic ideas and the mathematics of matrix factorization, and then we will present a simple implementation in Python. We will proceed with the assumption that we are dealing with user ratings (e.g. an integer score from the range of 1 to 5) of items in a recommendation system.

## 1. Basic Idea
Just as its name suggests, matrix factorization is to, obviously, factorize a matrix, i.e. to find out two (or more) matrices such that when you multiply them you will get back the original matrix.

As mentioned above, from an application point of view, matrix factorization can be used to discover latent features underlying the interactions between two different kinds of entities. (Of course, you can consider more than two kinds of entities and you will be dealing with tensor factorization, which would be more complicated.) And one obvious application is to predict ratings in collaborative filtering.

In a recommendation system such as Netflix or MovieLens, there is a group of users and a set of items (movies for the above two systems). Given that each users have rated some items in the system, we would like to predict how the users would rate the items that they have not yet rated, such that we can make recommendations to the users. In this case, all the information we have about the existing ratings can be represented in a matrix. Assume now we have 5 users and 10 items, and ratings are integers ranging from 1 to 5, the matrix may look something like this (a hyphen means that the user has not yet rated the movie):

<img src="https://cdn.mathpix.com/snip/images/HV42Io8ROOj5XSFpHLrs8AcxRr_k-Af_xptIJja1iOA.original.fullsize.png">

Hence, the task of predicting the missing ratings can be considered as filling in the blanks (the hyphens in the matrix) such that the values would be consistent with the existing ratings in the matrix.

The intuition behind using matrix factorization to solve this problem is that there should be some latent features that determine how a user rates an item. For example, two users would give high ratings to a certain movie if they both like the actors or actresses in the movie, or if the movie is an action movie, which is a genre preferred by both users.

Hence, if we can discover these latent features, we should be able to predict a rating with respect to a certain user and a certain item, because the features associated with the user should match with the features associated with the item.

In trying to discover the different features, we also make the assumption that the **number of features** would be smaller than the **number of users** and the **number of items.** It should not be difficult to understand this assumption because clearly it would not be reasonable to assume that each user is associated with a unique feature (although this is not impossible). And anyway if this is the case there would be no point in making recommendations, because each of these users would not be interested in the items rated by other users. Similarly, the same argument applies to the items.

## 2. Maths

Having discussed the intuition behind matrix factorization, we can now go on to work on the mathematics. Firstly, we have a set $U$ of users, and a set $D$ of items. Let $R$ of size $|U|×|D|$ be the matrix that contains all the ratings that the users have assigned to the items. Also, we assume that we would like to discover $K$ latent features. Our task, then, is to find two matrics matrices $P$ (of size $|U|×|K|$) and $Q$ (of size $|D|×|K|$) such that their product apprioximates $|R|$:

\begin{equation}
\mathbf{R} \approx \mathbf{P} \times \mathbf{Q}^{T}=\hat{\mathbf{R}}
\end{equation}

In this way, each row of $P$ would represent the strength of the associations between **a user and the features.** Similarly, each row of $Q$ would represent the strength of the associations between **an item and the features.** To get the prediction of a rating of an item ${d_j}$ by ${u_i}$, we can calculate the dot product of their vectors:

\begin{equation}
\hat{r}_{i j}=p_{i}^{T} q_{j}=\sum_{k=1}^{k} p_{i k} q_{k j}
\end{equation}

$K$ is user define

Now, we have to find a way to obtain $P$ and $Q$. One way to approach this problem is the first intialize the two matrices with some values, calculate how **different** their product is to $M$, and then try to minimize this difference iteratively. Such a method is called **gradient descent,** aiming at finding a local minimum of the difference.

The difference here, usually called the **error** between the estimated rating and the real rating, can be calculated by the following equation for each user-item pair:

\begin{equation}
e_{i j}^{2}=\left(r_{i j}-\hat{r}_{i j}\right)^{2}=\left(r_{i j}-\sum_{k=1}^{K} p_{i k} q_{k j}\right)^{2}
\end{equation}

Here we consider the **squared error** because the estimated rating can be either higher or lower than the real rating.

To minimize the error, we have to know in which direction we have to modify the values of ${p_ik}$ and ${q_kj}$. In other words, we need to know the gradient at the current values, and therefore we differentiate the above equation with respect to these two variables separately:

\begin{equation}
\begin{array}{l}{\frac{\partial}{\partial p_{i k}} e_{i j}^{2}=-2\left(r_{i j}-\hat{r}_{i j}\right)\left(q_{k j}\right)=-2 e_{i j} q_{k j}} \\ {\frac{\partial}{\partial q_{i k}} e_{i j}^{2}=-2\left(r_{i j}-\hat{r}_{i j}\right)\left(p_{i k}\right)=-2 e_{i j} p_{i k}}\end{array}
\end{equation}

Having obtained the gradient, we can now formulate the update rules for both ${p_ik}$ and ${q_kj}$:

\begin{equation}
\begin{array}{l}{p_{i k}^{\prime}=p_{i k}+\alpha \frac{\partial}{\partial p_{i k}} e_{i j}^{2}=p_{i k}+2 \alpha e_{i j} q_{k j}} \\ {q_{k j}^{\prime}=q_{k j}+\alpha \frac{\partial}{\partial q_{k j}} e_{i j}^{2}=q_{k j}+2 \alpha e_{i j} p_{i k}}\end{array}
\end{equation}

Here, $α$ is a constant whose value determines the rate of approaching the minimum. Usually we will choose a small value for $α$, say 0.0002. This is because if we make too large a step towards the minimum we may run into the risk of missing the minimum and end up oscillating around the minimum.

A question might have come to your mind by now: if we find two matrices $P$ and $Q$ such that $P×Q$ approximates $R$, isn’t that our predictions of all the unseen ratings will be zeros? In fact, we are not really trying to come up with $P$ and $Q$ such that we can reproduce $R$ exactly. Instead, we will only try to minimise the errors of the observed user-item pairs. In other words, if we let $T$ be a set of tuples, each of which is in the form of ${(u_i, d_j, r_ij)}$, such that $T$ contains all the observed user-item pairs together with the associated ratings, we are only trying to minimise every ${e_ij}$ for ${(u_i, d_j, r_ij )∈T}$. (In other words, $T$ is our set of training data.) As for the rest of the unknowns, we will be able to determine their values once the associations between the users, items and features have been learnt.

Using the above update rules, we can then iteratively perform the operation until the error converges to its minimum. We can check the overall error as calculated using the following equation and determine when we should stop the process.

\begin{equation}
E=\sum_{\left(u_{i}, d_{j}, r_{i j}\right) \in T} e_{i j}=\sum_{\left(u_{i}, d_{j}, r_{i j}\right) \in T}\left(r_{i j}-\sum_{k=1}^{K} p_{i k} q_{k j}\right)^{2}
\end{equation}


## 3. Regularization

The above algorithm is a very basic algorithm for factorizing a matrix. There are a lot of methods to make things look more complicated. A common extension to this basic algorithm is to introduce regularization to avoid **overfitting.** This is done by adding a parameter $β$ and modify the **squared error** as follows:

\begin{equation}
e_{i j}^{2}=\left(r_{i j}-\sum_{k=1}^{K} p_{i k} q_{k j}\right)^{2}+\frac{\beta}{2} \sum_{k=1}^{K}\left(\|P\|^{2}+\|Q\|^{2}\right)
\end{equation}


In other words, the new parameter $β$ is used to control the magnitudes of the user-feature and item-feature vectors such that $P$ and $Q$ would give a good approximation of $R$ without having to contain large numbers. In practice, $β$ is set to some values in the order of $0.02.$ The new update rules for this squared error can be obtained by a procedure similar to the one described above. The new update rules are as follows:

\begin{equation}
\begin{array}{l}{p_{i k}^{\prime}=p_{i k}+\alpha \frac{\partial}{\partial p_{i k}} e_{i j}^{2}=p_{i k}+\alpha\left(2 e_{i j} q_{k j}-\beta p_{i k}\right)} \\ {q_{k j}^{\prime}=q_{k j}+\alpha \frac{\partial}{\partial q_{i j}} e_{i j}^{2}=q_{k j}+\alpha\left(2 e_{i j} p_{i k}-\beta q_{k j}\right)}\end{array}
\end{equation}


## 4. Biases

When predicting the ratings of users given to items, it is useful to consider how ratings are generated. In the above discussion, we have assumed that ratings are generated based on matching the users preferences on some latent factors and the items’ characteristics on the latent factors. Actually, it may also be helpful to consider additional factors here.

For example, we can assume that when a rating is generated, some biases may also contribute to the ratings. In particular, every user may have his or her own bias, meaning that he or she may tend to rate items higher or lower than the others. In movie ratings, if a user is a serious movie watcher, he or she may tend to give lower ratings, when compared to another user who generally enjoys movies as long as they are not too boring. A similar idea can also be applied to the items being rated.

Hence, in the equal of predicting a rating, we can also add these biases in order to better model how a rating is generated:

\begin{equation}
\hat{r}_{i j}=b+b u_{i}+b d_{j}+\sum_{k=1}^{k} p_{i k} q_{k j}
\end{equation}

where $b$ is the global bias (which can be easily estimated by using the mean of all ratings), bui is the bias of user $i$, and ${bd_j}$ is the bias of item $j$.

Using the same steps mentioned above, we can derive the update rules for the user biases and item biases easily:

\begin{equation}
\begin{array}{l}{b u_{i}^{\prime}=b u_{i}+\alpha \times\left(e_{i j}-\beta b u_{i}\right)} \\ {b d_{j}^{\prime}=b d_{j}+\alpha \times\left(e_{i j}-\beta b d_{j}\right)}\end{array}
\end{equation}

In practice, the process of factorization will converge faster if biases are included in the model.